In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
import re
from copy import deepcopy
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor
ByteTensor = torch.ByteTensor


'''
这个项目例句:    'DESC:manner How did serfdom develop in and then leave Russia ?' 
转化为：   X= ['manner',' How ',' did ',' serfdom','  develop ',' in ',' and ',' then','  leave','  Russia','  ?']     y= 'desc'
转化为：   X= [234,14,1,41124,1414,1414]     y=5
X转化为   300维度的 wordvector
max_len 表示每个batch X最长的长度

(d,v) * (v,max_len,b) =(d, max_len ,b)

K=(3,4,5)  K表示strid     C_f 表示C_f个filter                       #三次不同filter size的卷积操作，然后用maxpool得到相同的size。然后concate
(d,max_len,b) 卷积操作_*   C_f个(d, K) =  (max_len-K+1,b,C_f)     K为3/4/5
然后三个(max_len-K+1,b,C_f)对于不同的max_len-K+1分别max_pool，得到三个(b,C_f)  
三个(b,C_f)  concate得到 (b,3*C_f)  
(out_size,3*C_f) * (b,3*C_f) = (out_size,b)

(out_size,b)  与b个y   softmax  得到结果
'''

"\n这个项目例句:    'DESC:manner How did serfdom develop in and then leave Russia ?' \n转化为：   X= ['manner',' How ',' did ',' serfdom','  develop ',' in ',' and ',' then','  leave','  Russia','  ?']     y= 'desc'\n转化为：   X= [234,14,1,41124,1414,1414]     y=5\nX转化为   300维度的 wordvector\nmax_len 表示每个batch X最长的长度\n\n(d,v) * (v,max_len,b) =(d, max_len ,b)\n\nK=(3,4,5)  K表示strid     C_f 表示C_f个filter                       #三次不同filter size的卷积操作，然后用maxpool得到相同的size。然后concate\n(d,max_len,b) 卷积操作_*   C_f个(d, K) =  (max_len-K+1,b,C_f)     K为3/4/5\n然后三个(max_len-K+1,b,C_f)对于不同的max_len-K+1分别max_pool，得到三个(b,C_f)  \n三个(b,C_f)  concate得到 (b,3*C_f)  \n(out_size,3*C_f) * (b,3*C_f) = (out_size,b)\n\n(out_size,b)  与b个y   softmax  得到结果\n"

In [2]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch
        
def pad_to_batch(batch):
    x,y = zip(*batch)
    max_x = max([s.size(1) for s in x])
    # print(max_x)
    x_p = []
    for i in range(len(batch)):
        if x[i].size(1) < max_x:
            x_p.append(torch.cat([x[i], Variable(LongTensor([word2index['<PAD>']] * (max_x - x[i].size(1)))).view(1, -1)], 1))
        else:
            x_p.append(x[i])
    return torch.cat(x_p), torch.cat(y).view(-1)


def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if to_index.get(w) is not None else to_index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

In [3]:
data = open(r'C:\workspace\python-work\nlp\NLP-cs224n\NLP-cs224n_ec\notebooks\dataset\cnn_classification\train_5500.labe.txt', 'r', encoding='latin-1').readlines()
#data[0]='DESC:manner How did serfdom develop in and then leave Russia ?' 
data = [[d.split(':')[1][:-1], d.split(':')[0]] for d in data]
#data[0] = ['manner How did serfdom develop in and then leave Russia ?',  'DESC']

X, y = list(zip(*data))
# print(X,'\n')
#  ('manner How did serfdom develop in and then leave Russia ?', 
#  'cremat What films featured the character Popeye Doyle ?'.......)

X = list(X)
# print(X)
#  ['manner How did serfdom develop in and then leave Russia ?', 
#  'cremat What films featured the character Popeye Doyle ?'.......]


for i, x in enumerate(X):
    X[i] = re.sub('\d', '#', x).split()
    if i in range(22,25):
        print(x,'========>',X[i])
   

ind Name 11 famous martyrs . ========> ['ind', 'Name', '##', 'famous', 'martyrs', '.']
desc What 's the Olympic motto ? ========> ['desc', 'What', "'s", 'the', 'Olympic', 'motto', '?']
desc What is the origin of the name ` Scarlett ' ? ========> ['desc', 'What', 'is', 'the', 'origin', 'of', 'the', 'name', '`', 'Scarlett', "'", '?']


In [4]:
vocab = list(set(flatten(X)))
print(len(vocab))
#X中单词的种类   9117

print('len(y)=',len(set(y)),'  set(y)=',set(y))
#y的种类
# len(y)= 6   set(y)= {'ABBR', 'LOC', 'NUM', 'HUM', 'ENTY', 'DESC'}

print(  set( [ i[0]    for i in X]))
#X数组中的第一个单词
#{'ord', 'word', 'product', 'sport', 'body', 'termeq', 'manner', 'volsize', 'code', 'weight', 'cremat', 'desc', 'veh', 'lang', 'food', 'ind', 'city', 'instru', 'count', 'substance', 'gr', 'letter', 'other', 'color', 'country', 'animal', 'state', 'religion', 'dismed', 'exp', 'period', 'currency', 'mount', 'money', 'symbol', 'def', 'plant', 'abb', 'event', 'title', 'speed', 'temp', 'reason', 'perc', 'dist', 'date', 'techmeth'}


9117
len(y)= 6   set(y)= {'ABBR', 'DESC', 'NUM', 'ENTY', 'LOC', 'HUM'}
{'techmeth', 'currency', 'food', 'product', 'veh', 'volsize', 'body', 'speed', 'letter', 'ord', 'animal', 'state', 'lang', 'color', 'mount', 'manner', 'exp', 'religion', 'date', 'symbol', 'sport', 'instru', 'dismed', 'title', 'substance', 'temp', 'other', 'abb', 'def', 'plant', 'dist', 'weight', 'gr', 'reason', 'termeq', 'city', 'code', 'event', 'count', 'country', 'cremat', 'ind', 'word', 'desc', 'period', 'money', 'perc'}


In [5]:
word2index={'<PAD>': 0, '<UNK>': 1}

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
index2word = {v:k for k, v in word2index.items()}

target2index = {}

for cl in set(y):
    if target2index.get(cl) is None:
        target2index[cl] = len(target2index)

index2target = {v:k for k, v in target2index.items()}

In [6]:
X_p, y_p = [], []
for pair in zip(X,y):
      #pair       (['ind', 'Name', '##', 'famous', 'martyrs', '.'],'DESC')
    X_p.append(prepare_sequence(pair[0], word2index).view(1, -1))             #append([ 234,14,1,41124,1414,1414   ])
    y_p.append(Variable(LongTensor([target2index[pair[1]]])).view(1, -1))
    
data_p = list(zip(X_p, y_p))       #  [ ([ 234,14,1,41124,1414,1414 ],5)   ,   ( [ 234,14,1,41124,1414,1414 ],5 ),..........]
random.shuffle(data_p)

train_data = data_p[: int(len(data_p) * 0.9)]
test_data = data_p[int(len(data_p) * 0.9):]

In [7]:
import gensim
#导入wordvect  ，300 dimention，3，000，000个单词
model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\workspace\python-work\nlp\NLP-cs224n\NLP-cs224n_ec\notebooks\dataset\cnn_classification\GoogleNews-vectors-negative300.bin', binary=True)

len(model.index2word)



3000000

In [8]:
pretrained = []
#将一维的word转化为300维的wordvector
for key in word2index.keys():
    try:
        pretrained.append(model[word2index[key]])
    except:
        pretrained.append(np.random.randn(300))
        
# print(pretrained[0])       #[ 124,24124,24...]  300d
pretrained_vectors = np.vstack(pretrained)

print(pretrained_vectors[0].shape)  #[ [ 124,24124,24...] ]  1*300d 
#(300,)
print(pretrained_vectors.shape)  #[ [ 124,24124,24...] ]  1*300d 

print(len(target2index))



(300,)
(9119, 300)
6


In [9]:
class  CNNClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, output_size, kernel_dim=100, kernel_sizes=(3, 4, 5), dropout=0.5):
        super(CNNClassifier,self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)      #(v,d)
        self.convs = nn.ModuleList(   [nn.Conv2d(1, kernel_dim, (K, embedding_dim))     for K in kernel_sizes]          )

        # kernal_size = (K,D) 
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes) * kernel_dim, output_size)  #(6,300)
    
    
    def init_weights(self, pretrained_word_vectors, is_static=False):
        self.embedding.weight = nn.Parameter(torch.from_numpy(pretrained_word_vectors).float())
        if is_static:
            self.embedding.weight.requires_grad = False


    def forward(self, inputs, is_training=False):
        inputs = self.embedding(inputs).unsqueeze(1)     # (d,v) * (v,max_length,B)  = (d,1,max_length,b)
        #print('1size=',inputs.size())                         # 1size= torch.Size([50, 1, 21, 300] )            (b,1,max_len,d)
        #下行对于每一个   (d,1,max_len,b) 卷积_*  （d,K) =  (max_len-K+1,b) 
        inputs = [F.relu(conv(inputs)).squeeze(3)    for conv in self.convs]   #[(b,C_filter,max_len-K+1 )*3 ]     #max_len=18,k=(3,4,5),C_filter=100,b=50
        # print("inputs[0].size=",inputs[0].size())      #inputs[0].size= torch.Size([50, 100, 16])
        # print("inputs[1].size=",inputs[1].size())      #inputs[1].size= torch.Size([50, 100, 15])
        # print("inputs[2].size=",inputs[2].size())      #inputs[2].size= torch.Size([50, 100, 14]) 
        inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs]       #[(b,C_filter)*3]
        # print("inputs[0].size=",inputs[0].size())      #inputs[0].size= torch.Size([50, 100])
        # print("inputs[1].size=",inputs[1].size())      # inputs[1].size= torch.Size([50, 100])
        # print("inputs[2].size=",inputs[2].size())        # inputs[2].size= torch.Size([50, 100])

        concated = torch.cat(inputs, 1)
        # print(concated.size())   #torch.Size([50, 300]) =  torch.Size([50, 100*3])

        if is_training:
            concated = self.dropout(concated) # (N,len(Ks)*Co)
        out = self.fc(concated)     #  (6,300)*(300,50)  = (6,50) = (out_size,batch)
        # print(out.size())   #torch.Size([50, 6])
        return F.log_softmax(out,1)
    
EPOCH = 1
BATCH_SIZE = 50
KERNEL_SIZES = [3,4,5]
KERNEL_DIM = 100
LR = 0.001

model = CNNClassifier(len(word2index), 300, len(target2index), KERNEL_DIM, KERNEL_SIZES)    #  (9117,300,6,[3,4,5],100)
print("model===========>",model)

'''
CNNClassifier(
  (embedding): Embedding(9119, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=6, bias=True)
)
'''
model.init_weights(pretrained_vectors) # initialize embedding matrix using pretrained vectors  (9119,300)


    
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)


for epoch in range(EPOCH):
    losses = []
    for i,batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        inputs,targets = pad_to_batch(batch)
        
        model.zero_grad()
        preds = model(inputs, True)  #input  (max_length,B,v)  
        # print(preds.size())    #log_softmax之后的   #(6,50) = (out_size,batch)
        loss = loss_function(preds, targets)
        losses.append(loss.data.tolist())
        loss.backward()
        
        #for param in model.parameters():
        #    param.grad.data.clamp_(-3, 3)
        
        optimizer.step()
        
        if i % 100 == 0:
            print("[%d/%d] mean_loss : %0.2f" %(epoch, EPOCH, np.mean(losses)))
            losses = []

model===========> CNNClassifier(
  (embedding): Embedding(9119, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=6, bias=True)
)


[0/1] mean_loss : 2.22


In [12]:
accuracy = 0
wrongn=0
#test_data =   [ ([ 234,14,1,41124,1414,1414 ],5)   ,   ( [ 234,14,1,41124,1414,1414 ],5 ),..........]
for i,test in enumerate(test_data):
    if(i==0):
        try:
            # print(i,test)f
            pred = model(test[0]).max(1)[1]
            print(1,pred)
            pred = pred.data.tolist()[0]
            print(2,pred)
            target = test[1].data.tolist()[0][0]
            print(target)
            if pred == target:
                accuracy += 1
            else:
                print(i)
        except:
            wrongn+=1
            print(wrong,"=>",i)
    # print('accuracy=',accuracy/len(test_data) * 100)

1 tensor([ 3])
2 3
3


In [17]:
accuracy = 0
wrongn=0
#test_data =   [ ([ 234,14,1,41124,1414,1414 ],5)   ,   ( [ 234,14,1,41124,1414,1414 ],5 ),..........]
for i,test in enumerate(test_data):
    if(i==0 ):
        try:
            # print(i,test)f
            pred = model(test[0])
            print(1,pred)
            pred = pred.data.tolist()[0]
            print(2,pred)
            target = test[1].data.tolist()[0][0]
            print(3,target)
            if pred == target:
                accuracy += 1
            else:
                print(i)
        except:
            wrongn+=1
            print(wrong,"=>",i)

1 tensor([[-3.2156, -4.8130, -2.5072, -0.4250, -1.5331, -7.3604]])
2 [-3.2156078815460205, -4.812989234924316, -2.507150173187256, -0.42504698038101196, -1.5330772399902344, -7.360416889190674]
3 3
0
